In [1]:
import pandas as pd

In [2]:
import sys
sys.path.insert(0, '..')

In [3]:
df = pd.read_csv("../src/datasets/data/simple-onlineshop.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35381 entries, 0 to 35380
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    35381 non-null  int64 
 1   event      35381 non-null  object
 2   timestamp  35381 non-null  object
dtypes: int64(1), object(2)
memory usage: 829.4+ KB


In [5]:
from src.eventstream import Eventstream, EventstreamSchema, RawDataSchema

/opt/homebrew/Caskroom/miniforge/base/envs/rete_pycharm/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
df['country'] = 'USA'
users = df['user_id'].unique()[:1500]
df.loc[df['user_id'].isin(users),'country'] = 'UK'

In [7]:
df.country.value_counts()

USA    18200
UK     17181
Name: country, dtype: int64

In [8]:
df.head()

,user_id,event,timestamp,country
0,219483890,catalog,2019-11-01 17:59:13.273932,UK
1,219483890,product1,2019-11-01 17:59:28.459271,UK
2,219483890,cart,2019-11-01 17:59:29.502214,UK
3,219483890,catalog,2019-11-01 17:59:32.557029,UK
4,964964743,catalog,2019-11-01 21:38:19.283663,UK


In [9]:
raw_data_schema=RawDataSchema(
                event_name="event",
                event_timestamp="timestamp",
                user_id="user_id",
                # custom_cols = [{"custom_col": 'country',
                #                  "raw_data_col": "country"}]
            )

In [10]:
df.head()

,user_id,event,timestamp,country
0,219483890,catalog,2019-11-01 17:59:13.273932,UK
1,219483890,product1,2019-11-01 17:59:28.459271,UK
2,219483890,cart,2019-11-01 17:59:29.502214,UK
3,219483890,catalog,2019-11-01 17:59:32.557029,UK
4,964964743,catalog,2019-11-01 21:38:19.283663,UK


In [13]:
source = Eventstream(
            raw_data_schema=raw_data_schema,
            raw_data=df)
source.to_dataframe(raw_cols=True)

,event_id,event_type,event_index,event,timestamp,user_id,raw_user_id,raw_event,raw_timestamp,raw_country
0,e66f40b1-1af8-4684-b6f8-a4fa1ad6a11d,raw,0,catalog,2019-11-01 17:59:13.273932,219483890,219483890,catalog,2019-11-01 17:59:13.273932,UK
1,e5431281-ce4c-4f54-b82b-82a9f8480aa6,raw,1,product1,2019-11-01 17:59:28.459271,219483890,219483890,product1,2019-11-01 17:59:28.459271,UK
2,916fc1cf-5c33-4654-b6ee-36064e13e5e1,raw,2,cart,2019-11-01 17:59:29.502214,219483890,219483890,cart,2019-11-01 17:59:29.502214,UK
3,5264b2e1-60c6-4634-a76f-784eed4a9f78,raw,3,catalog,2019-11-01 17:59:32.557029,219483890,219483890,catalog,2019-11-01 17:59:32.557029,UK
4,06103e2e-0b4b-4b22-84b0-bf8ae4e67553,raw,4,catalog,2019-11-01 21:38:19.283663,964964743,964964743,catalog,2019-11-01 21:38:19.283663,UK
...,...,...,...,...,...,...,...,...,...,...
35376,2dfaf927-0d99-4369-923a-c1fa6a7829bd,raw,35376,catalog,2020-04-29 12:47:40.975732,501098384,501098384,catalog,2020-04-29 12:47:40.975732,UK
35377,063c10c1-62b7-428f-af5b-66e9e92623a4,raw,35377,catalog,2020-04-29 12:48:01.809577,501098384,501098384,catalog,2020-04-29 12:48:01.809577,UK
35378,3c3621e8-277b-4f5f-8cb2-474b243d3393,raw,35378,main,2020-04-29 12:48:01.938488,501098384,501098384,main,2020-04-29 12:48:01.938488,UK
35379,00f1fc04-d41b-4c4f-9a70-691f9899c474,raw,35379,catalog,2020-04-29 12:48:06.595390,501098384,501098384,catalog,2020-04-29 12:48:06.595390,UK
